<a href="https://www.kaggle.com/code/uom190478e/notebook66e8b7766f?scriptVersionId=93273088" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/lab-02-introduction-to-machine-learning/sample_submission.csv
/kaggle/input/lab-02-introduction-to-machine-learning/train.csv
/kaggle/input/lab-02-introduction-to-machine-learning/test.csv


In [17]:
import numpy as np
import pandas as pd 
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestRegressor
import xgboost as xgb
from catboost import CatBoostRegressor
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.metrics import r2_score
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error

# **Load Datasets**

In [3]:
df_test = pd.read_csv('/kaggle/input/lab-02-introduction-to-machine-learning/test.csv')
df_train = pd.read_csv('/kaggle/input/lab-02-introduction-to-machine-learning/train.csv')

# **Data Preprocessing**

In [4]:
df = df_train.drop(['ID'] , axis=1)
df_test = df_test.drop(['ID'] , axis=1)

In [5]:
df = df.drop_duplicates()

In [6]:
def remove_outliers(p , features):
  df = p.copy()
  for f in features:
    count = 0
    q1 = df[f].quantile(0.25)
    q3 = df[f].quantile(0.75)
    iqr = q3-q1
    Lower_tail = q1 - 1.5 * iqr
    Upper_tail = q3 + 1.5 * iqr
    m = np.mean(df[f])
    df=df[~((df[f]<(Lower_tail)) | (df[f]>(Upper_tail)))]

  return df

In [7]:
features = list(df.columns)

In [8]:
df = remove_outliers(df , features)

In [9]:
X = df_train[['AT', 'V', 'AP', 'RH']]
Y = df_train.EP

In [10]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.3, random_state=42)

# **Model Development**

**Random Forest Regessor**

In [16]:
model1 = RandomForestRegressor()
model1.fit(X_train , y_train)
y_pred1 = model1.predict(X_test)

**Xgboost Regessor**

In [14]:
model2 = xgb.XGBRegressor()
model2.fit(X_train , y_train)
y_pred2 = model2.predict(X_test)

**Catboost Regessor**

In [15]:
model3 = CatBoostRegressor()
model3.fit(X_train , y_train)
y_pred3 = model3.predict(X_test)

Learning rate set to 0.054269
0:	learn: 16.1546008	total: 59ms	remaining: 58.9s
1:	learn: 15.4091856	total: 61.2ms	remaining: 30.5s
2:	learn: 14.6786287	total: 63.4ms	remaining: 21.1s
3:	learn: 13.9932476	total: 65.1ms	remaining: 16.2s
4:	learn: 13.3538510	total: 66.7ms	remaining: 13.3s
5:	learn: 12.7451390	total: 68.7ms	remaining: 11.4s
6:	learn: 12.1658357	total: 70.3ms	remaining: 9.97s
7:	learn: 11.6247264	total: 72.1ms	remaining: 8.94s
8:	learn: 11.1172677	total: 73.7ms	remaining: 8.12s
9:	learn: 10.6370603	total: 75.3ms	remaining: 7.46s
10:	learn: 10.1754991	total: 77.1ms	remaining: 6.93s
11:	learn: 9.7548348	total: 78.7ms	remaining: 6.48s
12:	learn: 9.3580413	total: 80.3ms	remaining: 6.1s
13:	learn: 8.9818858	total: 81.9ms	remaining: 5.76s
14:	learn: 8.6328717	total: 83.5ms	remaining: 5.48s
15:	learn: 8.3038568	total: 85.1ms	remaining: 5.23s
16:	learn: 8.0009528	total: 86.6ms	remaining: 5.01s
17:	learn: 7.7247302	total: 88.1ms	remaining: 4.81s
18:	learn: 7.4606468	total: 89.6ms	r

# # **Model Evaluation**

In [19]:
X_train.shape

(5950, 4)

In [36]:
def evaluate(y_test , y_pred):
    print("MAE",mean_absolute_error(y_test,y_pred) , '\n')
    
    print("MSE",mean_squared_error(y_test,y_pred), '\n')
    
    print("RMSE",np.sqrt(mean_squared_error(y_test,y_pred)), '\n')
    
    print("RMSLE",np.log(np.sqrt(mean_squared_error(y_test,y_pred))), '\n')
    
    r2 = r2_score(y_test,y_pred)
    print("R Squared",r2, '\n')
    
    Adj_r2 = 1-(1-r2)*(5950-1)/(5950-4-1)
    print("Adjusted R Squared",Adj_r2, '\n')

**Random Forest Regessor**

In [24]:
evaluate(y_test , y_pred1)

MAE 2.4260557254901913 

MSE 11.834938093243156 

RMSE 3.440194484799247 

RMSE 1.2355280060660427 

R Squared 0.9606270059196258 

Adjusted R Squared 0.960600514418142 



**Xgboost Regessor**

In [25]:
evaluate(y_test , y_pred2)

MAE 2.315032947974112 

MSE 10.735247912532698 

RMSE 3.2764688175736842 

RMSE 1.1867662624595088 

R Squared 0.9642855037194646 

Adjusted R Squared 0.9642614737808403 



**Catboost Regessor**

In [26]:
evaluate(y_test , y_pred3)

MAE 2.3778381151935437 

MSE 11.068343430992039 

RMSE 3.3269119962800398 

RMSE 1.202044545312898 

R Squared 0.9631773468560175 

Adjusted R Squared 0.9631525713114295 



# **Hyperparameter Tuning**

In [27]:
model_params = {
    'random_forest': {
        'model': RandomForestRegressor(),
        'params' : {
            'n_estimators':[75,100 , 125 , 150],
            'max_features':['auto', 'sqrt'],
            'max_depth':[int(x) for x in np.linspace(400, 2000, num = 20)],
            'min_samples_split':[5, 6, 7 , 8],
            'min_samples_leaf':[1, 2, 4],
            'bootstrap':[True, False]
        }
    },
     'xgboost': {
        'model': xgb.XGBRegressor(),
        'params':{
        'max_depth': [9 , 10 , 11],
        'n_estimators':[ 10000,9000],
        'learning_rate':[0.01 , 0.02 , 0.05],
        'min_child_weight':[0.4 , 0.5],
        'subsample':[0.5,0.6],
        'colsample_bytree':[0 , 1],
        'gamma':[0.3 , 0.4 , 0.5],
        }
    
    },
     'catboost': {
        'model': CatBoostRegressor(),
        'params' : {
            'depth':[8 , 9 , 10 , 11 , 12],
            'iterations':[3500 , 4500 , 5500],
            'learning_rate':[0.01 , 0.05 , 0.1]
        }
    }
}

**Hyperparameter tuning using gridsearch**

In [ ]:
# scores = []

# for model_name, mp in model_params.items():
#     print(model_name)
#     clf =  GridSearchCV(mp['model'], mp['params'], cv=3, return_train_score=False)
#     clf.fit(X,Y)
#     scores.append({
#         'model': model_name,
#         'best_score': clf.best_score_,
#         'best_params': clf.best_params_
#     })
#     print('done ' , model_name)
    
# df_tun = pd.DataFrame(scores,columns=['model','best_score','best_params'])
# df_tun

**Best Parameters**

In [28]:
best_params = {
    'random_forest': {
        'model': RandomForestRegressor(),
        'params' : {
            'n_estimators':[100],
            'max_features':['sqrt'],
            'max_depth':[450],
            'min_samples_split':[6],
            'min_samples_leaf':[1],
            'bootstrap':[False]
         }
     },
     'xgboost': {
        'model': xgb.XGBRegressor(),
        'params':{
            'max_depth': [10],
            'n_estimators':[ 9000],
            'learning_rate':[0.05],
            'min_child_weight':[0.5],
            'subsample':[0.5],
            'colsample_bytree':[1],
            'gamma':[0.5],
        }
    
    },
     'catboost': {
        'model': CatBoostRegressor(),
        'params' : {
            'depth':[10],
            'iterations':[5500],
            'learning_rate':[0.05 ]
        }
    }
}

**Random Forest Regessor**

In [29]:
model1 = RandomForestRegressor(bootstrap = False,
         max_depth = 450,
         max_features = 'sqrt',
         min_samples_leaf = 1,
         min_samples_split = 6,
         n_estimators = 100)

model1.fit(X_train , y_train)
y_pred1 = model1.predict(X_test)

In [32]:
evaluate(y_test , y_pred1)

MAE 2.291086967320261 

MSE 10.930996918345784 

RMSE 3.3062058191143793 

RMSE 1.195801253696438 

R Squared 0.9636342772925585 

Adjusted R Squared 0.9636098091864475 



**Xgboost Regessor**

In [30]:
model2 = xgb.XGBRegressor(colsample_bytree = 1 ,gamma = 0.5 ,  learning_rate = 0.05 , max_depth = 10 , min_child_weight = 0.5 , n_estimators = 9000 , subsample = 1 , eval_metric='mae')
model2.fit(X_train , y_train)
y_pred2 = model2.predict(X_test)

In [33]:
evaluate(y_test , y_pred2)

MAE 2.0942695278990504 

MSE 9.482280203222931 

RMSE 3.079331129193957 

RMSE 1.1247124075554484 

R Squared 0.9684539319624242 

Adjusted R Squared 0.9684327066853594 



**Catboost Regessor**

In [31]:
model3 = CatBoostRegressor(loss_function='RMSE' , depth=10,iterations=5500,learning_rate=0.05)
model3.fit(X_train , y_train)
y_pred3 = model3.predict(X_test)

0:	learn: 16.2102815	total: 17.5ms	remaining: 1m 36s
1:	learn: 15.4890927	total: 28.1ms	remaining: 1m 17s
2:	learn: 14.8171522	total: 51.9ms	remaining: 1m 35s
3:	learn: 14.1793268	total: 60.1ms	remaining: 1m 22s
4:	learn: 13.5653563	total: 69.2ms	remaining: 1m 16s
5:	learn: 12.9889527	total: 77.6ms	remaining: 1m 11s
6:	learn: 12.4364311	total: 88.6ms	remaining: 1m 9s
7:	learn: 11.9220854	total: 96.8ms	remaining: 1m 6s
8:	learn: 11.4316603	total: 105ms	remaining: 1m 3s
9:	learn: 10.9701359	total: 113ms	remaining: 1m 2s
10:	learn: 10.5209964	total: 121ms	remaining: 1m
11:	learn: 10.1027599	total: 129ms	remaining: 58.8s
12:	learn: 9.7169316	total: 137ms	remaining: 57.7s
13:	learn: 9.3477340	total: 144ms	remaining: 56.5s
14:	learn: 8.9910435	total: 156ms	remaining: 57.2s
15:	learn: 8.6529493	total: 164ms	remaining: 56.3s
16:	learn: 8.3514722	total: 171ms	remaining: 55.3s
17:	learn: 8.0495239	total: 179ms	remaining: 54.5s
18:	learn: 7.7722392	total: 187ms	remaining: 53.9s
19:	learn: 7.50821

In [34]:
evaluate(y_test , y_pred3)

MAE 2.099495212843638 

MSE 9.454887900182392 

RMSE 3.0748801440352747 

RMSE 1.1232659229493327 

R Squared 0.968545061884437 

Adjusted R Squared 0.9685238979227108 

